<a href="https://colab.research.google.com/github/Prabodh-09/SpeechRecognitionSystem/blob/main/Speech_Recognition_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import torchaudio.transforms as transforms

# Load the model and processor
model_name = "facebook/wav2vec2-large-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Function to transcribe audio
def transcribe_audio():
    # Take user input for the file path
    file_path = input("Enter the path to your audio file (any format): ")

    try:
        # Load the audio file (automatically handles format conversion)
        print("Loading audio file...")
        speech, rate = torchaudio.load(file_path)

        # Convert to 16kHz (Wav2Vec2 works best with 16kHz)
        if rate != 16000:
            print("Converting audio to 16kHz...")
            resampler = transforms.Resample(orig_freq=rate, new_freq=16000)
            speech = resampler(speech)

        # Limit the audio to 30 seconds (if longer, only the first 30 seconds are considered)
        max_duration = 30 * 16000  # 30 seconds * 16000 samples/sec
        speech = speech[:, :max_duration]

        # Prepare the audio for the model
        input_values = processor(speech.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values

        # Perform inference
        print("Transcribing audio...")
        logits = model(input_values).logits
        predicted_ids = logits.argmax(dim=-1)

        # Decode the predicted text
        transcription = processor.batch_decode(predicted_ids)[0]

        print("\nTranscription:")
        print(transcription)

    except Exception as e:
        print(f"Error: {e}\nPlease make sure the file path is correct and the format is supported.")

# Run the function
transcribe_audio()


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter the path to your audio file (any format): /content/harvard.wav
Loading audio file...
Converting audio to 16kHz...
Transcribing audio...

Transcription:
THE STALE SMELL OF OLD BEER LINGERS IT TAKES HEAT TO BRING OUT THE ODOUR A COLD DIP RESTORES HEALTH AND ZEST A SALT PICKLE TASTES FINE WITH HAM TAKO'S AL PASTOR ARE MY FAVORITE A ZESTFUL FOOD IS THE HOT CROSS BUN
